In [1]:
import sklearn
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [3]:
print(len(train))
train.head(10)

8693


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [4]:
print(len(test))
test.head()


4277


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [5]:
#fill nans
train = train.bfill()
test = test.bfill()

# drop strings, maybe considering tokenize for future tests
train = train.drop(["Name", "PassengerId"], axis=1)
                   
#save PassengerIDs for submission
IDs = test['PassengerId']
test = test.drop(["Name", "PassengerId"], axis=1)


# split up the Cabin format (deck/num/side) and add back to the train df
# num varies from 0 to over 1000
# side is either P or S
cabin = train.Cabin.str.split('/')
cabin = cabin.dropna()
cabin = cabin.tolist()
deck = [item[0] for item in cabin]
num = [item[1] for item in cabin]
side = [item[2] for item in cabin]

train = train.drop(columns=['Cabin'], axis=1)
train['Deck'] = deck
train['Num'] = num
train['Side'] = side
train.head()

# do the same for test df
cabin = test.Cabin.str.split('/')
cabin = cabin.tolist()
deck = [item[0] for item in cabin]
num = [item[1] for item in cabin]
side = [item[2] for item in cabin]

test = test.drop(columns=['Cabin'], axis=1)
test['Deck'] = deck
test['Num'] = num
test['Side'] = side
test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,G,3,S
1,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,F,4,S
2,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,C,0,S
3,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,C,1,S
4,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,F,5,S


In [6]:
# find unique values to encode for categorical features
print(train.HomePlanet.unique())
print(train.Destination.unique())
print(train.Deck.unique())

['Europa' 'Earth' 'Mars']
['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e']
['B' 'F' 'A' 'G' 'E' 'D' 'C' 'T']


In [7]:
# one hot encoding train and test sets for non-numerical, non-binary categorical features
planet = pd.get_dummies(train['HomePlanet'])
destination = pd.get_dummies(train['Destination'])
deck = pd.get_dummies(train['Deck'])
train = train.drop('HomePlanet',axis = 1)
train = train.drop('Destination',axis = 1)
train = train.drop('Deck',axis = 1)
train = train.join(planet)
train = train.join(destination)
train = train.join(deck)

planet = pd.get_dummies(test['HomePlanet'])
destination = pd.get_dummies(test['Destination'])
deck = pd.get_dummies(test['Deck'])
test = test.drop('HomePlanet',axis = 1)
test = test.drop('Destination',axis = 1)
test = test.drop('Deck',axis = 1)
test = test.join(planet)
test = test.join(destination)
test = test.join(deck)

# since Side is binary, we just convert to 0 or 1
train['Side'] = train['Side'].map({"S": 1, "P": 0})
test['Side'] = test['Side'].map({"S": 1, "P": 0})

# separate out Survived column to be our prediction data set
y_train = train["Transported"]
X_train = train.drop(["Transported"], axis=1)

print(X_train.head())

   CryoSleep   Age    VIP  RoomService  FoodCourt  ShoppingMall     Spa  \
0      False  39.0  False          0.0        0.0           0.0     0.0   
1      False  24.0  False        109.0        9.0          25.0   549.0   
2      False  58.0   True         43.0     3576.0           0.0  6715.0   
3      False  33.0  False          0.0     1283.0         371.0  3329.0   
4      False  16.0  False        303.0       70.0         151.0   565.0   

   VRDeck Num  Side  ...  PSO J318.5-22  TRAPPIST-1e  A  B  C  D  E  F  G  T  
0     0.0   0     0  ...              0            1  0  1  0  0  0  0  0  0  
1    44.0   0     1  ...              0            1  0  0  0  0  0  1  0  0  
2    49.0   0     1  ...              0            1  1  0  0  0  0  0  0  0  
3   193.0   0     1  ...              0            1  1  0  0  0  0  0  0  0  
4     2.0   1     1  ...              0            1  0  0  0  0  0  1  0  0  

[5 rows x 24 columns]


In [8]:
# Create a model
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
rfc = RandomForestClassifier()

# Fit the model to your training data.
rfc.fit(X_train, np.ravel(y_train))
y_pred = rfc.predict(test)

# build submission file
test["Transported"] = y_pred
test["PassengerID"] = IDs
submit = test[['PassengerID','Transported']]
submit

,PassengerID,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [9]:
submit.to_csv('submission.csv', index=False)